In [1]:
import tqdm
import os
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from torch import nn, optim
from torch.optim import Adam
from torch.utils.data import Dataset
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
from ipywidgets import interact
from rich.progress import Progress,TimeElapsedColumn,TimeRemainingColumn,BarColumn,TextColumn

from model_code import vdU_pred,SimVP
from data_struct import SegVid
from visualise import iou,gif_gen

In [4]:
train_data_seg = SegVid('train',1000)

100%|██████████| 1000/1000 [00:23<00:00, 42.23it/s]


In [5]:
val_data_seg = SegVid('val',1000)

100%|██████████| 998/998 [00:21<00:00, 45.48it/s]


# UnFixed video predictor

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = vdU_pred().to(device)

#first run
#model.load_weights('weights/SimVP_w','weights/UNet_w',device) 
#model.fix_vp() # do not comment

#optional later runs
model.load_state_dict(torch.load('weights/epoch500',map_location=device))

optim = Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.CrossEntropyLoss()

In [5]:
vp = SimVP((11,3,160,240)).to(device)
vp.load_state_dict(torch.load('weights/SimVP_w',map_location=device))

<All keys matched successfully>

In [8]:
i=0
if 'gifs' not in os.listdir():
    os.mkdir('gifs')
gif_gen(i,train_data_seg,'gifs/'+str(i),model,vp)

Original video(First 11 frames): ![SegmentLocal](gifs/1_orig.gif "segment")
Generaed video(Next 11 frames): ![SegmentLocal](gifs/1_simvp.gif "segment")
Generated video in sequence model: ![SegmentLocal](gifs/1_mod.gif "segment")